# Exploratory Analysis of Movie Data
### By: Christian Tsui


## Import Packages

In [160]:
# Your code here - remember to use markdown cells for comments as well!
# Connecting to the Database
import pandas as pd
import sqlite3



## Load & Preview Dataset

In [161]:
gross_csv = "./zippedData/bom.movie_gross.csv.gz"
imdb_zip = "./zippedData/im.db"
info_tsv = "./zippedData/rt.movie_info.tsv.gz"
reviews_tsv = "./zippedData/rt.reviews.tsv.gz"
tmdb_csv = "./zippedData/tmdb.movies.csv.gz"
budgets_csv = "./zippedData/tn.movie_budgets.csv.gz"

In [162]:
movie_gross_df = pd.read_csv(gross_csv)
movie_info_df = pd.read_csv(info_tsv, delimiter='\t')
movie_reviews_df = pd.read_csv(reviews_tsv, delimiter='\t', encoding='latin-1')
tmdb_df = pd.read_csv(tmdb_csv)
budgets_df = pd.read_csv(budgets_csv)

In [163]:
conn = sqlite3.connect(imdb_zip)

pd.read_sql("""SELECT name FROM sqlite_master WHERE type = 'table';""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


# Question 1: What genre of movies are most successful?

This question explores the idea of what genres of movies produce the most revenue relative to the size of the budget.

## Method:

### Inspecting the Gross Revenue Data

In [164]:
movie_gross_df[:10]

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [165]:
# As you can see there is missing data ('studio', 'domestic_gross', 'foreign_gross')
# 'foreign_gross' is also listen as an object, so I would like to convert it to a float
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### Cleaning the Gross Revenue Dataset

In [166]:
# Dropping missing values to maintain uifomity in the dataset
movie_gross_df.dropna(inplace=True)
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2007 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2007 non-null   float64
 3   foreign_gross   2007 non-null   object 
 4   year            2007 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 94.1+ KB


In [168]:
# Convert relavent data to float to provide consistency
# Removing punctuation in the string, converting to float
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross'].str.replace(',','').astype(float)

In [170]:
# Checking to see data types
movie_gross_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2007 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2007 non-null   float64
 3   foreign_gross   2007 non-null   float64
 4   year            2007 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 94.1+ KB


In [171]:
# Information shows the dataset is from year 2010 to 2018
movie_gross_df.describe()

,domestic_gross,foreign_gross,year
count,2.007000e+03,2.007000e+03,2007.000000
mean,4.701984e+07,7.579038e+07,2013.506228
std,8.162689e+07,1.381796e+08,2.597997
min,4.000000e+02,6.000000e+02,2010.000000
25%,6.700000e+05,3.900000e+06,2011.000000
50%,1.670000e+07,1.940000e+07,2013.000000
75%,5.605000e+07,7.595000e+07,2016.000000
max,9.367000e+08,9.605000e+08,2018.000000


In [174]:
# Looking at top 10 studios producing the most movies
movie_gross_df['studio'].value_counts().head(10)

Uni.     144
Fox      134
WB       130
Sony     105
BV       104
Par.      94
LGF       87
Wein.     69
IFC       68
SPC       59
Name: studio, dtype: int64

### Inspecting the Movie Budgets Data

In [175]:
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [176]:
# Displaying the data information, looking for missing values, data types
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [180]:
budgets_df['release_date'] = pd.to_datetime(budgets_df['release_date'])
budgets_df.head(3)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"


In [182]:
# extracting the relevant information
budgets_clean = budgets_df.loc[:,['release_date','movie','production_budget', 'domestic_gross', 'worldwide_gross']]

In [183]:
budgets_clean.head(3)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
